# Example 2: Computation of Gradients and Non-adiabatic Couplings

In this tutorial it is shown, how to use the SA-OO-VQE solver in a simple step-by-step manner on a molecule of
formaldimine (methylene imine). It is shown, how to compute energies, gradients and non-adiabatic couplings
in several different settings w.r.t. a different number of optimized orbitals.

First of all, we'll specify geometry of the system.

In [1]:
symbols = ['N', 'C', 'H', 'H', 'H']
coords = [
    [0.000000000000, 0.000000000000, 0.000000000000],
    [0.000000000000, 0.000000000000, 1.498047000000],
    [0.000000000000, -0.938765985000, 2.004775984000],
    [0.000000000000, 0.938765985000, 2.004775984000],
    [-0.744681452, -0.131307432, -0.634501434]
]

Now we'll specify its properties, active space and the basis for Psi4 chemistry backend.

In [2]:
n_orbs_active = 2
n_elec_active = 2
charge = 0
multiplicity = 1
basis = 'sto-3g'

The next step is construction of `ProblemSet` instance - object containing all the information and necessary method for
our electronic structure problem.

In [3]:
import saoovqe

problem = saoovqe.problem.ProblemSet(
    symbols=symbols,
    coords=coords,
    charge=charge,
    multiplicity=multiplicity,
    n_electrons_active=n_elec_active,
    n_orbitals_active=n_orbs_active,
    basis_name=basis
)

2024-09-11 18:37:04,977 - SAOOVQE.logger - INFO - SecondQuantizedProblem was created.


Now we need to create a set of circuits representing orthogonal states to construct the whole circuits representing
state vectors later.

In [4]:
initial_circuits = saoovqe.OrthogonalCircuitSet.from_problem_set(n_states=2, problem=problem)

2024-09-11 18:37:04,985 - SAOOVQE.logger - INFO - Circuits representing an orthogonal basis were created.


The next necessary part is to define an ansatz - it'll be also used to construct the state vector circuits later.

In [5]:
ansatz = saoovqe.Ansatz.from_problem_set(ansatz=saoovqe.AnsatzType.GUCCSD,
                                         problem=problem,
                                         repetitions=1,
                                         qubit_mapper=problem.fermionic_mapper)

2024-09-11 18:37:05,136 - SAOOVQE.logger - INFO - Ansatz was created.


And finally, now we can create an instance of our SA-OO-VQE solver. One of the main points is, orbital-optimization
can, but doesn't have to be used, or it can be used only on some molecular orbitals. We'll show all three cases here.
For no orbital-optimization it's enough to pass `None` to `orbital_optimization_settings` (it's also a default value).

In [6]:
from qiskit.primitives import Estimator, Sampler

estimator = Estimator()
sampler = Sampler()

solver_no_oo = saoovqe.SAOOVQE(estimator=estimator,
                               initial_circuits=initial_circuits,
                               ansatz=ansatz,
                               problem=problem,
                               orbital_optimization_settings=None)

2024-09-11 18:37:05,141 - SAOOVQE.logger - INFO - SAOOVQE was created.


To specify number of optimized orbitals we can pass a dictionary to the parameter.

In [7]:
solver_oo_8 = saoovqe.SAOOVQE(estimator=estimator,
                              initial_circuits=initial_circuits,
                              ansatz=ansatz,
                              problem=problem,
                              orbital_optimization_settings={'n_mo_optim': 8})

2024-09-11 18:37:05,148 - SAOOVQE.logger - INFO - SAOOVQE was created.


And for all the orbitals to be optimized we can simply use its default behavior by passing an empty dictionary.

In [8]:
solver_oo_full = saoovqe.SAOOVQE(estimator=estimator,
                                 initial_circuits=initial_circuits,
                                 ansatz=ansatz,
                                 problem=problem,
                                 orbital_optimization_settings={})

2024-09-11 18:37:05,153 - SAOOVQE.logger - INFO - SAOOVQE was created.


Let's compare the energies now! To make the numerical optimizations, we'll use `SLSQP` optimizer provided by `SciPy`.
It may take a few minutes time now...

In [9]:
from qiskit_algorithms.optimizers import SciPyOptimizer
import numpy as np

optimizer = SciPyOptimizer('SLSQP', options={'maxiter': 500, 'ftol': 1e-8})
energies_no_oo = solver_no_oo.get_energy(optimizer)
energies_oo_8 = solver_oo_8.get_energy(optimizer)
energies_oo_full = solver_oo_full.get_energy(optimizer)

print('\n============== State-Averaged Energies ==============')
print(np.mean(energies_no_oo))
print(np.mean(energies_oo_8))
print(np.mean(energies_oo_full))

2024-09-11 18:37:05,159 - SAOOVQE.logger - INFO - Computing energies...
2024-09-11 18:37:06,468 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.27262107  0.27261257 -0.01552155]
2024-09-11 18:37:06,637 - SAOOVQE.logger - INFO - Optimal phi angle for state-resolution was obtained (phi* = 3.806882565485043).
2024-09-11 18:37:07,069 - SAOOVQE.logger - INFO - Computing energies...
2024-09-11 18:37:08,164 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.27262107  0.27261257 -0.01552155]
2024-09-11 18:37:08,165 - SAOOVQE.logger - INFO - Starting Orbital-Optimization process...
2024-09-11 18:37:10,826 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.25184031  0.25183824 -0.0144197 ]
2024-09-11 18:37:10,827 - SAOOVQE.logger - INFO - Starting Orbital-Optimization process...
2024-09-11 18:37:13,314 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.2398365   0.23983421 -0.01432425]
2024-09-11 18:37:13,315 - SAOOVQE.logger - INFO - Starting Orbit


============== State-Averaged Energies ==============
-92.6425659307685
-92.64414484313264
-92.65981061303728


And now, let's have a look at the gradients of the potential energy surface for all the particles at both relevant
states.

In [10]:
print('\n============== Gradients ==============')
for state_idx in range(2):
    for atom_idx in range(len(symbols)):
        print(state_idx, atom_idx, solver_oo_full.eval_eng_gradient(state_idx, atom_idx))

2024-09-11 18:39:05,966 - SAOOVQE.logger - INFO - Constructing gradient evaluators...



============== Gradients ==============
[array([0.1594999 , 0.15941297, 0.00392104]), array([-0.15975057, -0.15964672, -0.00392346])]
0 0 [-0.22489509 -0.04953975 -0.20136349]
[array([0.1594999 , 0.15941297, 0.00392104]), array([-0.15975057, -0.15964672, -0.00392346])]
0 1 [0.07450464 0.01798229 0.23643613]
[array([0.1594999 , 0.15941297, 0.00392104]), array([-0.15975057, -0.15964672, -0.00392346])]
0 2 [-0.00249513  0.02463421 -0.02604222]
[array([0.1594999 , 0.15941297, 0.00392104]), array([-0.15975057, -0.15964672, -0.00392346])]
0 3 [-0.00682759 -0.02588031 -0.02034392]
[array([0.1594999 , 0.15941297, 0.00392104]), array([-0.15975057, -0.15964672, -0.00392346])]
0 4 [0.15971316 0.03280356 0.0113135 ]
[array([0.1594999 , 0.15941297, 0.00392104]), array([-0.15975057, -0.15964672, -0.00392346])]
1 0 [-0.0410096  -0.02400584 -0.54280118]
[array([0.1594999 , 0.15941297, 0.00392104]), array([-0.15975057, -0.15964672, -0.00392346])]
1 1 [0.06493983 0.00099405 0.51643413]
[array([0.159499

And the non-adiabatic couplings.

In [11]:
print('\n============== Total non-adiabatic couplings ==============')
for atom_idx in range(len(symbols)):
    print(atom_idx, solver_oo_full.eval_nac(atom_idx))


============== Total non-adiabatic couplings ==============
0 [-0.55768231  2.37527181  1.29031111]
1 [-0.12649155 -1.09991067 -1.32599432]
2 [0.010722   0.01867561 0.22103471]
3 [ 0.27471929  0.08494871 -0.10622852]
4 [ 0.36415499 -1.27159027 -0.07587189]


And finally, considering NACs, we can also have a look at CI and CSF NACs separately.

In [12]:
print('\n============== CI non-adiabatic couplings ==============')
for atom_idx in range(len(symbols)):
    print(atom_idx, solver_oo_full.ci_nacs[atom_idx])

print('\n============== CSF non-adiabatic couplings ==============')
for atom_idx in range(len(symbols)):
    print(atom_idx, solver_oo_full.csf_nacs[atom_idx])


============== CI non-adiabatic couplings ==============
0 [-0.51633172  2.1620211   1.29018778]
1 [-0.11974115 -1.06874766 -1.32508811]
2 [-0.03526327  0.02012146  0.22513544]
3 [ 0.31514566  0.08634093 -0.11030773]
4 [ 0.35619048 -1.19973583 -0.07992739]

============== CSF non-adiabatic couplings ==============
0 [-4.13505881e-02  2.13250719e-01  1.23333343e-04]
1 [-0.0067504  -0.03116302 -0.00090621]
2 [ 0.04598526 -0.00144585 -0.00410074]
3 [-0.04042637 -0.00139221  0.00407921]
4 [ 0.00796451 -0.07185444  0.0040555 ]
